In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

width = 12
height = 7
plt.rcParams["figure.figsize"] = (width, height)


from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import tflscripts
import json
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from statistics import mode

In [2]:
activities = [
    "Dishes",
    "Microwave",
    "Coffee",
    "Null",
    "Kettle",
    "Chopping food",
    "Conversation",
    "Eating popcorn",
    "Knocking",
    "Phone vibrating",
    "Microwave button press",
    "Microwave door opened",
    "Microwave door closed"
]

configuration = tflscripts.read_configuration()
activities_i = [configuration['activities'].index(a) for a in activities]

tflscripts.set_dataset_folder('/home/giotto/transfer-learning-playground/datasets/')

use_features = "^(?!mag|light)"

In [7]:
import pickle

class TestResult:
    def __init__(self,

                 source_device=None,
                 source_dataset=None,
                 
                 target_device=None,
                 target_dataset=None,

                 label=None,
                 columns=None,

                 predicted=None,
                 actual=None,
                 actual_with_all_labels=None,
                 window_size=None,
                 classifier=None,

                 features=None):
        self.source_device = source_device
        self.source_dataset = source_dataset
        self.target_device = target_device
        self.target_dataset = target_dataset

        self.label = label
        self.columns = columns
        self.window_size = window_size
        self.features = features
        self.classifier = classifier

        self.predicted = predicted
        self.actual = actual
        self.actual_with_all_labels = actual_with_all_labels

    def label_name(self):
        return configuration['activities'][self.label]


class TestSet:
    def __init__(self, name):
        self.name = name
        self.file_name = self.name + '.pkl'


    def add_result(self, result):
        output = open(self.file_name, 'ab')
        pickle.dump(result, output)
        output.close()
        print('Added result for', result.target_device, ' in ', result.target_dataset)


    def get_results(self):
        input = open(self.file_name, 'rb')
        results = []
        try:
            while True:
                result = pickle.load(input)
                results.append(result)

        except (EOFError) as e:
            input.close()

        return results

In [5]:
training_devices = [
    ['synergy-final-iter1', '128.237.254.195', 'Synergy 1 sink'], # sink
    ['synergy-final-iter1', '128.237.246.127', 'Synergy 1 coffee'], # coffee
    ['synergy-final-iter1', '128.237.248.186', 'Synergy 1 table'], # table
    ['synergy-final-iter2', '128.237.248.186', 'Synergy 2 sink'], # sink
    ['synergy-final-iter2', '128.237.254.195', 'Synergy 2 coffee'], # coffee
    ['synergy-final-iter2', '128.237.246.127', 'Synergy 2 table'], # table
#     ['synergy-final-iter3', '128.237.237.122', 'Synergy 3 sink'], # sink
#     ['synergy-final-iter3', '128.237.239.234', 'Synergy 3 coffee'], # coffee
#     ['synergy-final-iter3', '128.237.234.0', 'Synergy 3 table'], # table
    ['scott-final-iter1', '128.237.248.186', 'Scott 1 left'], # left
    ['scott-final-iter1', '128.237.247.134', 'Scott 1 right'], # right
    ['robotics-final', '128.237.246.127', 'Robotics coffee'], # coffee
    ['robotics-final', '128.237.247.134', 'Robotics sink'], # sink
    ['robotics-final', '128.237.248.186', 'Robotics entrance'], # entrance
]
testing_devices = [
    ['synergy-final-iter1', '128.237.254.195', 'Synergy 1 sink'], # sink
    ['synergy-final-iter1', '128.237.246.127', 'Synergy 1 coffee'], # coffee
    ['synergy-final-iter1', '128.237.248.186', 'Synergy 1 table'], # table
    ['synergy-final-iter2', '128.237.248.186', 'Synergy 2 sink'], # sink
    ['synergy-final-iter2', '128.237.254.195', 'Synergy 2 coffee'], # coffee
    ['synergy-final-iter2', '128.237.246.127', 'Synergy 2 table'], # table
#     ['synergy-final-iter3', '128.237.237.122', 'Synergy 3 sink'], # sink
#     ['synergy-final-iter3', '128.237.239.234', 'Synergy 3 coffee'], # coffee
#     ['synergy-final-iter3', '128.237.234.0', 'Synergy 3 table'], # table
    ['scott-final-iter1', '128.237.248.186', 'Scott 1 left'], # left
    ['scott-final-iter1', '128.237.247.134', 'Scott 1 right'], # right
    ['robotics-final', '128.237.246.127', 'Robotics coffee'], # coffee
    ['robotics-final', '128.237.247.134', 'Robotics sink'], # sink
    ['robotics-final', '128.237.248.186', 'Robotics entrance'], # entrance
]
pipelines = []

features_to_use = [
    '.*',
    'MICROPHONE',
    'ACCEL',
    'EMI',
    'MICROPHONE|ACCEL_'
]

classifiers = [
    'SVM',
    'RandomForestClassifier',
    'LogisticRegression'
]


test_set = TestSet(name='testing')


for source in training_devices:
    source_dataset = source[0]
    source_device = source[1]

    df_source, df_source_labels = tflscripts.read_and_filter_dataset(
            source_dataset + '-1s',
            source_device,
            use_features='.*',
            use_activities=activities_i,
            scale=True,
            with_feature_selection=False)

    df_source = df_source.loc[df_source.index.isin(df_source_labels.index)]
    df_source_labels = df_source_labels.loc[df_source_labels.index.isin(df_source.index)]

    for label in df_source_labels.label.unique():
        for features in features_to_use:
            x_train = df_source.filter(regex=features)
            y_train = tflscripts.get_y_for_label(df_source_labels, label)
            
            for classifier in classifiers:
                if classifier == 'SVM':
                    clf = svm.SVC(kernel='linear', decision_function_shape='ovr')
                elif classifier == 'RandomForestClassifier':
                    clf = RandomForestClassifier()
                elif clf_name == 'LogisticRegression':
                    clf = LogisticRegression()

                ppl = Pipeline([
                    ('impute', Imputer()),
                    ('clf', clf)
                ])

                ppl.fit(x_train, y_train)


                for target in testing_devices:                
                    target_dataset = target[0]
                    target_device = target[1]

                    if source_dataset == target_dataset and source_device == target_device:
                        continue

                    label_pipelines = {}

                    df_target, df_target_labels = tflscripts.read_and_filter_dataset(
                        target_dataset + '-1s',
                        target_device,
                        use_features='.*',
                        use_activities=activities_i,
                        scale=True,
                        with_feature_selection=False)

                    df_target = df_target.loc[df_target.index.isin(df_target_labels.index)]

                    y_test = tflscripts.get_y_for_label(df_target_labels, label)
                    x_test = df_target[x_train.columns]

                    predicted = ppl.predict(x_test)
                    result = TestResult(
                        source_dataset=source_dataset,
                        source_device=source_device,

                        target_dataset=target_dataset,
                        target_device=target_device,

                        predicted=predicted,
                        actual=y_test.values,
                        actual_with_all_labels=df_target_labels.label.values,

                        classifier=classifier,
                        window_size='1s',
                        label=label,
                        columns=x_train.columns,
                        features=features
                    )

                    test_set.add_result(result)
            break
        break
    break

Added result for 128.237.246.127  in  synergy-final-iter1
Added result for 128.237.248.186  in  synergy-final-iter1
Added result for 128.237.248.186  in  synergy-final-iter2
Added result for 128.237.254.195  in  synergy-final-iter2
Added result for 128.237.246.127  in  synergy-final-iter2
Added result for 128.237.248.186  in  scott-final-iter1
Added result for 128.237.247.134  in  scott-final-iter1
Added result for 128.237.246.127  in  robotics-final
Added result for 128.237.247.134  in  robotics-final
Added result for 128.237.248.186  in  robotics-final
Added result for 128.237.254.195  in  synergy-final-iter1
Added result for 128.237.248.186  in  synergy-final-iter1
Added result for 128.237.248.186  in  synergy-final-iter2
Added result for 128.237.254.195  in  synergy-final-iter2
Added result for 128.237.246.127  in  synergy-final-iter2
Added result for 128.237.248.186  in  scott-final-iter1
Added result for 128.237.247.134  in  scott-final-iter1
Added result for 128.237.246.127  in 

KeyboardInterrupt: 

In [10]:
test_set = TestSet(name='testing')
results = test_set.get_results()